# experiments with DuckDB and geoparquet

This notebook explores the possibility of working with the SnowEx data in geoparquet format rather than using a PostgreSQL database. The advantage of this approach would be that we could host the data on S3 without the need for a persistent database management service.

## Links:

* [DuckDB](https://duckdb.org/)
* [geoparquet](https://geoparquet.org/)
* [geoparquet tutorials](https://github.com/cholmes/duckdb-geoparquet-tutorials)
* [duckdb spatial mapping](https://www.google.com/search?q=duckdb+plot+spatial&oq=duckdb+plot+spatial&gs_lcrp=EgZjaHJvbWUyBggAEEUYOTIKCAEQABiABBiiBDIKCAIQABiABBiiBDIKCAMQABiABBiiBDIKCAQQABiABBiiBDIKCAUQABiABBiiBNIBCDMyNTNqMGoxqAIAsAIA&sourceid=chrome&ie=UTF-8#fpstate=ive&vld=cid:75dcf93e,vid:r3A_OLT-JBM,st:0)


## Converting from postgresql

As an initial test, let's convert one of the main snowexsql database tables to geoparquet (the 'points' table). Then we can compare query times with our existing API.

To do this I followed the steps in [this documentation](https://www.crunchydata.com/blog/parquet-and-postgres-in-the-data-lake).

Details:
* ```mamba install conda-forge::gdal```
* ```mamba install -c conda-forge libgdal-pg```
* ```mamba install -c conda-forge libdgal-arrow-parquet```
* ``` ogr2ogr -f Parquet /tmp/points.parquet PG:"dbname='snowex' host='db.snowexdata.org' port='5432' user='snow' password='hackweek'" points```

Next I uploaded the points.parquet file to an S3 bucket and acquired a security key for accessing the file.

In [1]:
import duckdb
!pip install geoparquet # this is not yet on cryocloud
!pip install python-dotenv
import duckdb
import pandas as pd
import geopandas as gpd
import geoparquet as gpq
from dotenv import load_dotenv
from snowexsql.api import PointMeasurements
import os
from dotenv import dotenv_values

### enable the aws and httpfs extensions in duckdb

In [2]:
con = duckdb.connect()
con.install_extension("httpfs")
con.load_extension("httpfs")
con.install_extension("aws")
con.load_extension("aws")

In [3]:
load_dotenv() # credentials come from a local .env file
k=os.getenv('KEY_ID')
s=os.getenv('SECRET')

In [5]:
duckdb.sql("""CREATE SECRET (TYPE S3,KEY_ID {}, SECRET {}, REGION 'us-west-2')""".format(k,s))

┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ true    │
└─────────┘

### Try showing all the columns of the database

In [10]:
duckdb.sql("DESCRIBE SELECT * FROM read_parquet('s3://snowexdb/points.parquet')")

┌────────────────┬────────────────────────────────────────────────────────┬─────────┬─────────┬─────────┬─────────┐
│  column_name   │                      column_type                       │  null   │   key   │ default │  extra  │
│    varchar     │                        varchar                         │ varchar │ varchar │ varchar │ varchar │
├────────────────┼────────────────────────────────────────────────────────┼─────────┼─────────┼─────────┼─────────┤
│ id             │ BIGINT                                                 │ YES     │ NULL    │ NULL    │ NULL    │
│ site_name      │ VARCHAR                                                │ YES     │ NULL    │ NULL    │ NULL    │
│ date           │ DATE                                                   │ YES     │ NULL    │ NULL    │ NULL    │
│ time_created   │ TIMESTAMP WITH TIME ZONE                               │ YES     │ NULL    │ NULL    │ NULL    │
│ time_updated   │ TIMESTAMP                                            

## Simple query to replicate the first example in the db tutorial

In [11]:
df = duckdb.sql("SELECT value, longitude, latitude FROM read_parquet('s3://snowexdb/points.parquet') WHERE type='depth' AND instrument='pit ruler' LIMIT 100").to_df()

In [12]:
# Convert DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(
    df, 
    geometry=gpd.points_from_xy(df.longitude, df.latitude)
)

# Set the coordinate reference system (CRS) if needed
gdf.set_crs(epsg=4326, inplace=True)

,value,longitude,latitude,geometry
0,83.0,-108.06311,39.04496,POINT (-108.06311 39.04496)
1,100.0,-108.19593,39.04563,POINT (-108.19593 39.04563)
2,117.0,-108.14791,39.00760,POINT (-108.14791 39.0076)
3,98.0,-108.16401,39.02144,POINT (-108.16401 39.02144)
4,92.0,-108.19103,39.03404,POINT (-108.19103 39.03404)
...,...,...,...,...
95,92.0,-108.20975,39.03596,POINT (-108.20975 39.03596)
96,35.0,-108.18948,39.03126,POINT (-108.18948 39.03126)
97,101.0,-108.15596,39.01843,POINT (-108.15596 39.01843)
98,102.0,-108.14158,39.01437,POINT (-108.14158 39.01437)


In [13]:
gdf.explore()

In [21]:
df = duckdb.sql("SELECT value, latitude, longitude FROM read_parquet('s3://snowexdb/layers.parquet') WHERE type='specific_surface_area'").to_df()

In [22]:
# Convert DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(
    df, 
    geometry=gpd.points_from_xy(df.longitude, df.latitude)
)

# Set the coordinate reference system (CRS) if needed
gdf.set_crs(epsg=4326, inplace=True)

,value,latitude,longitude,geometry
0,36.0,39.053236,-108.200765,POINT (-108.20076 39.05324)
1,27.1,39.053236,-108.200765,POINT (-108.20076 39.05324)
2,24.3,39.053236,-108.200765,POINT (-108.20076 39.05324)
3,21.0,39.053236,-108.200765,POINT (-108.20076 39.05324)
4,20.7,39.053236,-108.200765,POINT (-108.20076 39.05324)
...,...,...,...,...
2169,12.1,39.029403,-108.145964,POINT (-108.14596 39.0294)
2170,12.6,39.029403,-108.145964,POINT (-108.14596 39.0294)
2171,10.0,39.029403,-108.145964,POINT (-108.14596 39.0294)
2172,11.0,39.029403,-108.145964,POINT (-108.14596 39.0294)


In [23]:
gdf.explore()